# But du code est de retrouver l'algorithme quantique de Hallgren permettant de résoudre l'équation de Pell x²-Dy²=1.


### Introduction : Résolution de l'Équation de Pell

L'équation de Pell s'écrit sous la forme x²−Dy²=1, où D est un entier non carré parfait. Résoudre cette équation revient à trouver l'unité fondamentale du corps quadratique réel Q(sqrt(D)​).

L'algorithme de Hallgren utilise le fait que les solutions sont liées à la périodicité d'une fonction définie sur les idéaux réduits du corps. La période de cette fonction est appelée le Régulateur (R). Une fois R trouvé, on peut en extraire x et y.

## Étape 0 : Configuration et Imports


In [ ]:
import math
import numpy as np
from fractions import Fraction
from decimal import Decimal, getcontext
getcontext().prec = 100 # Haute précision pour Pell

# Qiskit et simulation
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, transpile
from qiskit.circuit.library import QFT
from qiskit.quantum_info import Operator
from qiskit.visualization import plot_histogram
from qiskit_aer import AerSimulator

### **Étape 1 : Le Moteur Arithmétique (Classique)**

Le but de cette étape est de définir les fondations mathématiques nécessaires pour manipuler les objets liés à l'équation de Pell.

* **Représentation des Idéaux :** Utilisation d'une classe `Ideal` pour représenter les idéaux réduits du corps quadratique, définis par des paramètres () et une distance logarithmique.
* **Algorithme de Réduction (`step_rho`) :** Implémentation de l'algorithme 2.1 qui permet de passer d'un idéal au suivant dans un cycle en calculant sa nouvelle distance.
* **Calcul du Régulateur :** Une fonction permet de trouver classiquement le régulateur  (la période du cycle) en itérant jusqu'à revenir à l'idéal unité ().

### **Étape 2 : L'Oracle Quantique (Fonction périodique)**

Cette étape prépare l'opérateur quantique qui "encode" la périodicité du régulateur.

* **Discrétisation du Cycle :** On pré-calcule les idéaux réduits tout au long du cycle pour créer une base de données d'états.
* **Construction des Sauts (`Jumps`) :** L'oracle est construit en utilisant l'exponentiation binaire. Il applique des matrices de permutation (sauts contrôlés) qui déplacent un état d'une distance donnée sur le cycle : .
* **Garantie de l'Unitarité :** Les matrices de saut sont conçues pour être des permutations uniques afin de respecter les lois de la mécanique quantique.

### **Étape 3 : L'Algorithme Quantique (Circuit de Période)**

Il s'agit du cœur de la résolution, utilisant une structure similaire à l'algorithme de Shor.

* **Mise en Superposition :** Application de portes Hadamard sur le registre de contrôle pour explorer toutes les distances possibles simultanément.
* **Appel à l'Oracle :** Application des sauts contrôlés calculés à l'étape précédente.
* **Transformée de Fourier Quantique Inverse (IQFT) :** On applique une IQFT pour faire émerger la fréquence (et donc la période ) de la superposition.
* **Mesure :** On mesure le registre de contrôle pour obtenir une valeur , qui est une approximation liée au régulateur.

### **Étape 4 : Post-traitement et Recherche des Solutions**

L'information brute mesurée doit être transformée pour obtenir le résultat final.

* **Développement en Fractions Continues :** Utilisation de la valeur mesurée pour calculer les convergents de la fraction  (où  est la taille du registre).
* **Récupération du Régulateur :** À partir des dénominateurs de ces convergents, on extrait des candidats pour le régulateur .
* **Vérification des Candidats :** Pour chaque candidat, on vérifie s'il est un multiple du régulateur en cherchant si l'idéal à cette distance est l'idéal unité. Cela permet de confirmer la solution de l'équation de Pell.

## Étape 1 : Le Moteur Arithmétique (Classique)

Cette partie repose sur l'infrastructure des fractions continues de D​. On représente un idéal par un triplet (a,b,d) correspondant au nombre b+sqrt(d)​/a​.

In [ ]:
class Ideal:
    """
    Représente un idéal réduit dans le cycle associé à sqrt(D).
    Un idéal est défini par (a, b) tel que (b + sqrt(d))/a est une fraction continue.
    """
    def __init__(self, a, b, d, distance=0.0):
        self.a = int(a)
        self.b = int(b)
        self.d = d
        self.distance = distance # Logarithme de la norme (utilisé pour la période)

    def __repr__(self):
        return f"Ideal(a={self.a}, b={self.b}, dist={self.distance:.6f})"


def step_rho(ideal):
    """
    Opérateur de 'Baby-step' (ρ).
    Calcule l'idéal réduit suivant dans le cycle à partir de l'actuel.
    C'est une étape de l'algorithme des fractions continues.
    """
    a, b, d = ideal.a, ideal.b, ideal.d
    sqrt_d = math.sqrt(d)
    
    # Formules classiques de réduction des formes quadratiques / fractions continues
    q = int((b + sqrt_d) / a)
    b_next = q * a - b
    a_next = (d - b_next**2) // a
    
    # La distance augmente de log(theta), où theta est le nombre associé à l'étape
    diff_dist = math.log(abs((b + sqrt_d) / a))
    return Ideal(a_next, b_next, d, distance=ideal.distance + diff_dist)


def get_classical_regulator(d):
    """
    Calcule le régulateur R de manière classique pour servir de référence.
    On itère ρ jusqu'à revenir à l'idéal principal (a=1).
    """
    # L'idéal unité commence avec a=1
    curr = Ideal(1, int(math.sqrt(d)), d, 0.0)
    # On limite à 1000 itérations pour éviter les boucles infinies sur de grands D
    for _ in range(1000):
        curr = step_rho(curr)
        if curr.a == 1 and curr.distance > 0:
            return curr.distance
    return None


# Test du moteur classique
D_test = 7
R_true = get_classical_regulator(D_test)
print(f"Régulateur classique pour D={D_test} (R) : {R_true}")

## Étape 2 : L'Oracle Quantique (Fonction périodique)

L'objectif de l'oracle est d'implémenter la fonction périodique f(x). Dans l'algorithme de Hallgren, cette fonction associe à un nombre réel x un couple (I,δ) où :
- I est l'idéal réduit tel que sa distance au premier idéal est la plus grande distance inférieure ou égale à x.
- δ=x−dist(I) est l'écart résiduel.

Pour la simulation, nous discrétisons le cycle des idéaux. L'oracle va effectuer des "sauts" (jumps) sur ce cycle.

### 2.1 Fonctions de support (Discrétisation et Recherche)

Nous devons d'abord pouvoir naviguer dans le cycle de manière classique pour construire nos opérateurs quantiques.

In [ ]:
def precompute_discretized_cycle(d, R):
    """
    Génère la liste exhaustive des idéaux réduits sur une période R.
    Ces idéaux représentent les 'états' de notre registre f(x).
    """
    states = []
    curr = Ideal(1, int(math.sqrt(d)), d, 0.0)
    # On parcourt le cycle jusqu'à atteindre la période R
    while curr.distance < R:
        states.append(curr)
        curr = step_rho(curr)
    return states

def find_closest_state_index(states, target_dist, R):
    """
    Trouve l'indice de l'idéal dont la distance est la plus proche de target_dist (modulo R).
    """
    target_dist = target_dist % R
    # Calcul des distances circulaires pour trouver le plus proche voisin
    distances = [abs(s.distance - target_dist) for s in states]
    # Prise en compte du bouclage en fin de cycle (distance vers R)
    dist_to_end = abs(R - target_dist)
    
    if dist_to_end < min(distances):
        return 0 # On est plus proche de l'origine (Idéal unité)
    return np.argmin(distances)

### 2.2 Génération des Opérateurs de Saut (Jumps)

L'oracle de Hallgren repose sur des sauts contrôlés. Si le bit j du registre de contrôle est à 1, on avance d'une distance proportionnelle à 2j.

In [ ]:
def compute_jump_matrix(states, jump, R):
    """
    Crée une matrice unitaire de permutation représentant un saut de distance 'jump'.
    U|Ideal_i> = |Ideal_{i + jump}>
    """
    n_states = len(states)
    n_qubits = int(np.ceil(np.log2(n_states)))
    dim = 2**n_qubits
    
    # Initialisation d'une matrice d'identité de taille 2^n
    matrix = np.zeros((dim, dim), dtype=complex)
    
    used_targets = set()
    for i in range(n_states):
        # Calcul de la destination théorique
        target_dist = (states[i].distance + jump) % R
        target_idx = find_closest_state_index(states, target_dist, R)
        
        # Pour garantir l'unitarité (permutation parfaite), on évite les collisions
        # dues à la discrétisation
        while target_idx in used_targets:
            target_idx = (target_idx + 1) % n_states
            
        matrix[target_idx, i] = 1.0
        used_targets.add(target_idx)
    
    # Remplissage des états hors-cycle (padding) avec l'identité
    for i in range(n_states, dim):
        matrix[i, i] = 1.0
        
    return matrix

### 2.3 Assemblage de l'Oracle Quantique

Cette fonction construit le circuit final en utilisant l'exponentiation binaire, comme dans l'algorithme de Shor.

In [ ]:
def build_hallgren_oracle(n_ctrl, d, T, R_approx):
    """
    Construit le circuit de l'oracle.
    n_ctrl   : Nombre de qubits de précision (registre de contrôle)
    d        : Discriminant
    T        : Borne supérieure de recherche (généralement > R)
    R_approx : Estimation du régulateur pour construire la simulation
    """
    # 1. Préparation des états du cycle
    cycle_states = precompute_discretized_cycle(d, R_approx)
    n_f_qubits = int(np.ceil(np.log2(len(cycle_states))))
    
    # 2. Création des registres
    qr_x = QuantumRegister(n_ctrl, name='x')        # Registre de contrôle (la distance)
    qr_f = QuantumRegister(n_f_qubits, name='f_x')  # Registre cible (l'idéal)
    qc = QuantumCircuit(qr_x, qr_f)

    # 3. Application des sauts binaires contrôlés
    for j in range(n_ctrl):
        # La distance de saut double à chaque bit : (2^j / 2^n) * T
        # Cela permet d'encoder toutes les distances possibles en superposition
        jump_distance = (2**j / (2**n_ctrl)) * T
        
        # Calcul de la matrice et conversion en opérateur Qiskit
        op_matrix = compute_jump_matrix(cycle_states, jump_distance, R_approx)
        jump_gate = Operator(op_matrix).to_instruction()
        jump_gate.name = f"Jump_2^{j}"
        
        # Application contrôlée par le bit j du registre x
        qc.append(jump_gate.control(), [qr_x[j]] + list(qr_f))
        
    return qc

## Étape 3 : Circuit de Période (Approximation de Hallgren)

Cette étape est une adaptation de l'algorithme de recherche de période de Shor. La différence majeure réside dans le fait que le régulateur R est un nombre réel (souvent irrationnel). Nous utilisons donc un registre de contrôle pour échantillonner une fonction périodique sur un intervalle [0,T].

### 3.1 Architecture du Circuit

Le circuit se compose de trois segments :
- Initialisation : On place le registre de contrôle en superposition totale pour tester toutes les distances simultanément.
- L'Oracle de Saut : On applique les opérateurs unitaires Uj​ qui effectuent des "jumps" sur le cycle des idéaux. C'est ici que la périodicité du régulateur est "imprimée" dans la phase du registre de contrôle.
- La Transformée de Fourier Quantique Inverse (IQFT) : Elle permet de passer du domaine des distances au domaine des fréquences pour faire apparaître les pics de probabilité correspondant à R.

### 3.2 Implémentation du Circuit


In [ ]:
# --- PARAMÈTRES DE CONFIGURATION ---
d = 2
n_ctrl = 8          # Nombre de qubits de contrôle (définit la précision)
R_approx = get_classical_regulator(d)    # Référence classique pour la simulation
T = 2 * R_approx    # Fenêtre d'échantillonnage (doit être > R)

# 1. Pré-calcul des états pour la simulation
cycle_states = precompute_discretized_cycle(d, R_approx)
n_state_qubits = int(np.ceil(np.log2(len(cycle_states))))

# 2. Initialisation des registres
qr_ctrl = QuantumRegister(n_ctrl, 'ctrl')
qr_tgt = QuantumRegister(n_state_qubits, 'ideal')
cr = ClassicalRegister(n_ctrl, 'result')
qc = QuantumCircuit(qr_ctrl, qr_tgt, cr)

# --- CONSTRUCTION ---

# A. Superposition (Hadamard sur le registre de contrôle)
# On crée une superposition de tous les états de 0 à 2^n_ctrl - 1
qc.h(qr_ctrl)
qc.barrier()

# B. Application de l'Oracle (Sauts contrôlés)
# On utilise l'exponentiation binaire pour l'efficacité
for j in range(n_ctrl):
    # Distance du saut : chaque qubit j représente une fraction de T
    jump_distance = (2**j / (2**n_ctrl)) * T
    
    # Génération de la matrice unitaire de saut
    op_matrix = compute_jump_matrix(cycle_states, jump_distance, R_approx)
    gate_Uj = Operator(op_matrix).to_instruction()
    gate_Uj.name = f"Jump_2^{j}"
    
    # Application contrôlée : si ctrl[j] == 1, on saute de jump_distance
    qc.append(gate_Uj.control(), [qr_ctrl[j]] + list(qr_tgt))

qc.barrier()

# C. QFT Inverse et Mesure
# On transforme les phases en valeurs mesurables (fréquences)
qc.append(QFT(n_ctrl).inverse(), qr_ctrl)
qc.measure(qr_ctrl, cr)

print(f"Circuit construit : {n_ctrl} qubits de contrôle, {n_state_qubits} qubits d'état.")
# qc.draw("mpl")

### Étape 3.3 : Simulation et Analyse des Fréquences

Dans cette phase, nous exécutons le circuit. Les résultats les plus fréquents (les "pics") nous donnent des valeurs c qui satisfont approximativement l'équation :
c/2**n_(ctrl)​≈{k⋅T/R​}

Où {⋅} désigne la partie fractionnaire et k est un entier.

In [ ]:
# 1. Configuration du simulateur
simulator = AerSimulator()

# 2. Transpilation (Optimisation pour le simulateur)
compiled_circuit = transpile(qc, simulator)

# 3. Exécution
# Un nombre élevé de 'shots' est nécessaire pour filtrer le bruit dû à l'irrégularité du cycle
shots = 4096
job = simulator.run(compiled_circuit, shots=shots)
result = job.result()
counts = result.get_counts()

# 4. Analyse des résultats
sorted_counts = dict(sorted(counts.items(), key=lambda item: item[1], reverse=True))

print(f"Analyse des {shots} simulations :")
print("-" * 40)
for i, (state, count) in enumerate(list(sorted_counts.items())[:5]):
    measured_int = int(state, 2)
    probability = (count / shots) * 100
    print(f"Top {i+1}: Valeur {measured_int:3} | Probabilité: {probability:>5.2f}% | Binaire: {state}")

# 5. Visualisation des pics de probabilité
plot_histogram(counts, title=f"Histogramme des mesures (D={d}, T={T:.2f})")

## Étape 4 : Post-traitement (Fractions Continues)

### Étape 4.1 : Analyse Quantique (Extraction du Régulateur)

C'est ici que l'on nettoie le résultat. Le circuit de Hallgren (comme celui de Shor) nous donne des pics de fréquence. Si la fenêtre de simulation est T, et que le cycle (le Régulateur R) se répète c fois dedans, alors on mesurera la valeur c.

La relation clé est donc :
R≈T​/c

In [ ]:
def get_regulator_candidates(counts, T):
    """
    Convertit les mesures quantiques en estimations du régulateur R.
    """
    # On récupère les pics triés (déjà fait en 3.3, mais on assure le coup ici)
    sorted_items = sorted(counts.items(), key=lambda item: item[1], reverse=True)
    
    candidates = []
    
    # On analyse les 3 pics principaux
    for bitstring, count in sorted_items[:3]:
        c = int(bitstring, 2)
        
        # Le pic à 0 correspond à la composante continue (fréquence nulle), on l'ignore
        if c == 0:
            continue
            
        probability = count / sum(counts.values())
        
        # FORMULE FONDAMENTALE : R = T / Fréquence
        # Si c est le nombre de cycles dans la fenêtre T, alors la longueur d'un cycle est T/c
        R_est = T / c
        
        print(f"Pic détecté : c={c:<3} (Prob: {probability:.1%})")
        print(f"   -> Hypothèse fondamentale (k=1) : R ≈ {T:.4f} / {c} = {R_est:.5f}")
        
        candidates.append(R_est)
        
        # Optionnel : On pourrait ajouter les harmoniques (R * 2, etc), 
        # mais avec D=7 et ce circuit, le fondamental est généralement dominant.

    return candidates

# Exécution
candidates_R = get_regulator_candidates(counts, T)

### Étape 4.2 : Retour au Classique (Trouver x et y)

C'est l'étape finale. On a une valeur floue de R (ex: 2.761). On doit trouver les entiers x,y tels que x2−Dy2=1. Au lieu de chercher l'idéal réduit (complexe), on peut utiliser directement la formule qui lie le régulateur à la solution :
x+ysqrt(D)​=eR

D'où l'on tire :
x=cosh(R) et y=sinh(R)​/sqrt(D)

In [ ]:
def solve_pell_classic(candidates, d):
    """
    Cherche la solution entière (x,y) à partir des estimations de R.
    """
    # On utilise Decimal pour gérer la précision si D est grand
    getcontext().prec = 50 
    d_dec = Decimal(d)
    sqrt_d = d_dec.sqrt()
    
    found = False
    for R_val in candidates:
        print(f"Test du candidat R ≈ {R_val:.5f}...")
        
        # 1. Calcul des valeurs théoriques de x et y basées sur R
        val_exp = Decimal(R_val).exp()
        
        # x = cosh(R) et y = sinh(R)/sqrt(D)
        x_calc = (val_exp + 1/val_exp) / 2
        y_calc = (val_exp - 1/val_exp) / (2 * sqrt_d)
        
        # 2. Arrondi à l'entier le plus proche
        x = int(x_calc.to_integral_value())
        y = int(y_calc.to_integral_value())
        
        # 3. Vérification rigoureuse
        # Attention : pour x=1, y=0 c'est la solution triviale, on l'ignore souvent
        if y == 0: 
            print("   -> Solution triviale (y=0), on continue.")
            continue

        res = x**2 - d * y**2
        
        if res == 1:
            print(f"   ✅ SUCCÈS ! Solution trouvée.")
            print(f"   -> x = {x}")
            print(f"   -> y = {y}")
            print(f"   -> Vérification : {x}² - {d}×{y}² = 1")
            found = True
            break # On s'arrête dès qu'on a trouvé
        else:
            print(f"   ❌ Échec (x²-Dy² = {res})")
    
    if not found:
        print("\n⚠️ Aucune solution exacte trouvée. Essayez d'augmenter la précision du simulateur (plus de qubits) ou la taille de la fenêtre T.")

# Lancement final avec les variables globales d et candidates_R
solve_pell_classic(candidates_R, d)